In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'onepercent:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5700058%2F9392584%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240919%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240919T205939Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Da83c995fe54b9beab29f4c23f0b7068de4cdeaa11f3a17ccc132bf77b44a568c9a19c2a1ff41cd872032e06527d67075e58b445b98aafb5563f8068fc2d3a66a39fb7e9a33e836803fdb6379bf1d5daa438f5b6aefed39fb48fea879c55e40914b3294de8b26537cab48467d6adf38559b5555b51b0c7bf96231f89bd5c2a3567ed1cb34fae0b9aa775fe3cf0e7d0b4b369a3cd50a47a46a780c5e15e03559ec7004286d04de85a6ad50dca661bdd86537c4a4806aea0ede7ba16935f012cb49f9fc1a042039c2f2e87d59b83065481fbfeaa83437366d77df32a25b126afd27f69de62cd688eeb6372d639b1760066a81a254082d2a0ed6d2e39520082e28ba,excludeclasses:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5702947%2F9396311%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240919%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240919T205939Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D764314a37660774f9e980c6fa886450f9ec2c891ed9fcbf889673302e2ad673816d16b257cbbd2be5ac6edee99ace21d76efa787938f8aad1860d2892e244293ba52240e6d51cceb34fd33efaa5708969b89d0520e6506497e84180abaad90af3be9ed42f1fa3ca0251b7aefe7565ba8374fcc2710bbbc243e4157a51f945574d76798dcf63a5f6a3f95c3aed08561fabe8779aa617fa4d501afa42a4db42a76ccad08b0a4d9a68c669bdb02ed0fb7cfa891e0decd793673b2377b3e1d2e9bf14078e8b86ddbe4823142cef813b81ffe0b282d394dc5ee98cc8f99d0d0b7402192702e8b080e8467aa6f5b086a7d367cee48204072c91cb694263413899e9b70,yolomlchallenge:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5710250%2F9405396%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240919%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240919T205939Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D2a8088eecbefc5b5495bff068571a3908342197c0d6b6930270212e02ac6d58444ee63654c098411ea9a44c218dcdd24b62a5836cdb9a0db9f9e43e035e3cfacaa794e3cf5043c59b2883584ca5ec6fa45ccef5ec6e2fd4d12ef6f6281d63639c9c9b39689ba2d720a3da8138b611ed04c65ce8ff16d2ee44f4e58115e980000a837bc309af33f567df423f16ef29b04b4671000a5a643d30b96cba3eeaeba3e416c54ddbfe26d9b52dbfaf88a3845d85a4e30f5066b9e9b53b5fdecbb7547de26c2c294f4866b6f37faef5b82d2bbd89033616804322e0c069ce030ab2870bb722e533173798a7e0b7751692eb2aed79aee522b846dc432665e3375c997d98a,testml:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5712052%2F9407591%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240919%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240919T205940Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Db114d82e3087f2c04f153c1fd1c22598d8952116f9cb4e8fa5b623f9e84863c39c6de0c7f72967a2de561beed8f054c2215c60e15e07e2143ba0e359c63ba0515d3dd02abfdd2c188857e68298d01024936b7cb02068f9c1fc26dd7539eeb9832824c9e94eb4d5ee0a6d24e255f11252fd52e25953d3a8fe4113497ad1a449b08adfc71bd382047c5a65b52e633f19862fa20f973a546e62745256c7c52295786035942e8cbd9428109a469448a9110928991e04c2b68beb15b3606c2f3fce75bb03d3bc9efe08df064830aaea5379214f146f6fbf28fb8fdc469bda14bee42ee0a88a6e61bbd07030e88635e7168f2214c20ec361c84f313e39bea439096106'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 30805 bytes downloaded
Downloaded and uncompressed: onepercent
[==================================================] 1834796 bytes downloaded
Downloaded and uncompressed: excludeclasses
[==================================================] 180083676 bytes downloaded
Downloaded and uncompressed: yolomlchallenge
[==================================================] 1542536 bytes downloaded
Downloaded and uncompressed: testml
Data source import complete.


In [41]:
import numpy as np
import pandas as pd
import requests
from io import BytesIO
import re
from PIL import Image
import logging

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/excludeclasses/acc.csv
/kaggle/input/onepercent/onepercent.csv
/kaggle/input/yolomlchallenge/yolobest.pt
/kaggle/input/testml/test.csv


In [3]:
!pip install paddlepaddle
!pip install paddleocr
!pip install --upgrade paddlepaddle-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.5/407.5 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 104.9 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.6.0-py2.py3-none-any.whl size=117030 sha256=840a5b2a4bede4901b9431cfb95198bd0361b7741ab01ce0a4ca108bcbe20e6d
  Stored in 

In [4]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 873.6/873.6 kB 45.3 MB/s eta 0:00:00


In [5]:
from paddleocr import PaddleOCR, draw_ocr

In [6]:
df = pd.read_csv('/kaggle/input/onepercent/onepercent.csv')
# df = pd.read_csv('/kaggle/input/testml/test.csv')

In [7]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
Tesla T4


In [8]:
ocr = PaddleOCR(use_angle_cls=True, lang='en', use_gpu=True, max_batch_size=50, debug=False)
#Initializing the model with language - English, Batch_size - 50, Angular Text Support - True

download https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_det_infer.tar to /root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer/en_PP-OCRv3_det_infer.tar


100%|██████████| 4.00M/4.00M [00:00<00:00, 5.17MiB/s]


download https://paddleocr.bj.bcebos.com/PP-OCRv4/english/en_PP-OCRv4_rec_infer.tar to /root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer/en_PP-OCRv4_rec_infer.tar


100%|██████████| 10.2M/10.2M [00:01<00:00, 8.63MiB/s]


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to /root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer/ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████| 2.19M/2.19M [00:01<00:00, 2.01MiB/s]

[2024/09/19 21:02:50] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=True, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=50, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_ch

In [9]:
from ultralytics import YOLO
model = YOLO('/kaggle/input/yolomlchallenge/yolobest.pt')

WARNING ⚠️ /kaggle/input/yolomlchallenge/yolobest.pt appears to require 'dill', which is not in Ultralytics requirements.
AutoInstall will run now for 'dill' but this feature will be removed in the future.
Recommend fixes are to train a new model using the latest 'ultralytics' package or to run a command with an official Ultralytics model, i.e. 'yolo predict model=yolov8n.pt'
requirements: Ultralytics requirement ['dill'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 66.0 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 2.3s, installed 1 package: ['dill']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



In [10]:
def download_image(image_url):
    try:
        response = requests.get(image_url, timeout=10)
        response.raise_for_status()
        image = Image.open(BytesIO(response.content)).convert('RGB')
        return image
    except requests.HTTPError as e:
        print(f"HTTP error occurred while downloading image from {image_url}: {e}")
        return None
    except IOError as e:
        print(f"IO error occurred while opening image from {image_url}: {e}")
        return None

In [11]:
def croppedOCR(image, box):
    cropped_image = image.crop((box[0], box[1], box[2], box[3])) # Crop the image to the bounding box
    image_np = np.array(cropped_image) # Convert cropped image to NumPy array
    return ocr.ocr(image_np) # Perform OCR on the cropped region

In [12]:
def extract_value_and_unit_by_keyword(result, keyword):
    patterns = {
        'item_weight': r'(\d+(?:\.\d+)?)(?:V)?-?(\d+(?:\.\d+)?)*\s*(kg|g|lbs|lb|Ib|pounds|pound|oz|mcg|t|ton|mg|gram|grams|kgs|lbm)',
        'height': r'(\d+(?:\.\d+)?)(?:V)?-?(\d+(?:\.\d+)?)*\s*(cm|mm|in|inch|inches|m|meter|meters|ft|feet|foot|yd|yard|yards)',
        'width': r'(\d+(?:\.\d+)?)(?:V)?-?(\d+(?:\.\d+)?)*\s*(cm|mm|in|inch|inches|m|meter|meters|ft|feet|foot|yd|yard|yards)',
        'depth': r'(\d+(?:\.\d+)?)(?:V)?-?(\d+(?:\.\d+)?)*\s*(cm|mm|in|inch|inches|m|meter|meters|ft|feet|foot|yd|yard|yards)',
        'voltage': r'(\d+(?:\.\d+)?)(?:V)?-?(\d+(?:\.\d+)?)*\s*(volt|volts|v|V|kV|kilovolt|kilovolts)',
        'wattage': r'(\d+(?:\.\d+)?)(?:V)?-?(\d+(?:\.\d+)?)*\s*(watt|watts|w|W|kilowatt|kilowatts|kW|megawatt|megawatts|MW)',
        'item_volume': r'(\d+(?:\.\d+)?)(?:V)?-?(\d+(?:\.\d+)?)*\s*(cl|ft|foot|feet|c|in|inch|inches|dl|fl oz|fluid ounces|FL oz|gal|gallon|gallons|imp gal|imperial gallon|l|L|liter|liters|ml|mL|milliliter|milliliters|pt|pint|pints|qt|quart|quarts|ul|microliter|mcl|cup|cups)',
        'maximum_weight_recommendation': r'(\d+(?:\.\d+)?)(?:V)?-?(\d+(?:\.\d+)?)*\s*(kg|g|lbs|lb|pounds|pound|oz|mcg|t|ton|mg|gram|grams|kgs|lbm)'
    }

    pattern = patterns.get(keyword.lower())
    if not pattern:
        return "", None

    if result is None:
        return "", None

    # Ensure result is a list of lists
    if not isinstance(result, list) or not all(isinstance(line, list) for line in result):
        return "", None

    for line in result:
        for element in line:
            if not isinstance(element, list) or len(element) < 2:
                continue
            text = element[1][0]
            matches = re.findall(pattern, text, re.IGNORECASE)
            if matches:
                if matches[0][1]:  # Handle ranges if applicable
                    range_values = f"[{float(matches[0][0])}, {float(matches[0][1])}] {matches[0][2]}"
                    return range_values, element[1][1]
                else:
                    return f"{float(matches[0][0])} {matches[0][2]}", element[1][1]

    return "", None  # Returning an empty string when no result is fetched

### Function to detect dimensions if the entity_name is from one of Height / Width / Depth

In [44]:
def detect_with_yolo(image, entity_name):
    yolo_results = model(image)

    if not yolo_results:
        logging.warning("No YOLO results returned.")
        return None

    yolo_result = yolo_results[0]  # Assuming batch size 1

    class_map = {'height': 'height', 'width': 'width', 'depth': 'depth'}
    relevant_class = class_map.get(entity_name.lower())

    if not relevant_class:
        logging.warning(f"No relevant class mapping for entity '{entity_name}'.")
        return None

    boxes = []
    # Access detections in yolo_result.boxes
    detections = yolo_result.boxes  # boxes is a Boxes object
    if detections is None or len(detections) == 0:
        logging.info(f"No detections found in image for entity '{entity_name}'.")
        return None

    for box in detections:
        # box.cls is a tensor containing the class id
        class_id = int(box.cls.item())  # Correct extraction of class_id
        class_name = model.names[class_id]
        if class_name.lower() == relevant_class.lower():
            # box.xyxy is a tensor containing [x1, y1, x2, y2]
            bbox = box.xyxy.tolist()[0]  # Convert tensor to list
            boxes.append(bbox)

    if not boxes:
        logging.info(f"No bounding boxes found for class '{relevant_class}' in image.")
        return None

    return boxes  # List of bounding boxes

In [48]:
# Process the images and add extracted values to the submission DataFrame
def process_images_for_submission(df):
    predictions = []

    for index, row in df.iterrows():
        if index % 100 == 0:
            print("Processing Row", index)

        image_url = row['image_link']
        entity_name = row['entity_name']

        # Download the image
        image = download_image(image_url)
        if image is None:
            predictions.append("")
            continue

        # If entity is height, width, or depth, use YOLO to detect bounding box
        if entity_name.lower() in ['height', 'width', 'depth']:
            bounding_boxes = detect_with_yolo(image, entity_name)
            if bounding_boxes is None:
                predictions.append("")  # No relevant box found, return empty prediction
                continue

            # Run OCR on each bounding box
            extracted_value = ""
            for box in bounding_boxes:
                result = croppedOCR(image, box)
                if result is not None:
                    value, _ = extract_value_and_unit_by_keyword(result, entity_name)
                    if value:
                        extracted_value = value
                        break  # Stop after finding the first relevant value

            predictions.append(extracted_value if extracted_value else "")
        else:
            # For other entity names, perform OCR on the entire image
            image_np = np.array(image)  # Convert PIL Image to NumPy array
            result = ocr.ocr(image_np)
            if result is None:
                predictions.append("")
                continue

        # Extract value from image
        extracted_value, _ = extract_value_and_unit_by_keyword(result, entity_name)
        predictions.append(extracted_value)

    # Create a new DataFrame with 'index' and 'prediction' columns
    # submission_df = pd.DataFrame({
    #     'index': df['index'],
    #     'prediction': predictions
    # })

    # return submission_df
    df['prediction'] = predictions

    return df

# Process the test images and generate the submission file
submission_df = process_images_for_submission(df)
submission_df.to_csv('submissions1.csv', index=False)  # Save the result to a new CSV file


Streaming output truncated to the last 5000 lines.
[2024/09/19 22:15:12] ppocr DEBUG: cls num  : 13, elapsed : 0.03086996078491211
[2024/09/19 22:15:12] ppocr DEBUG: rec_res num  : 13, elapsed : 0.06319499015808105
[2024/09/19 22:15:12] ppocr DEBUG: dt_boxes num : 46, elapsed : 0.0778343677520752
[2024/09/19 22:15:12] ppocr DEBUG: cls num  : 46, elapsed : 0.0691988468170166
[2024/09/19 22:15:12] ppocr DEBUG: rec_res num  : 46, elapsed : 0.12676382064819336
[2024/09/19 22:15:12] ppocr DEBUG: dt_boxes num : 2, elapsed : 0.0572507381439209
[2024/09/19 22:15:13] ppocr DEBUG: cls num  : 2, elapsed : 0.008422374725341797
[2024/09/19 22:15:13] ppocr DEBUG: rec_res num  : 2, elapsed : 0.01255345344543457
[2024/09/19 22:15:13] ppocr DEBUG: dt_boxes num : 19, elapsed : 0.05490374565124512
[2024/09/19 22:15:13] ppocr DEBUG: cls num  : 19, elapsed : 0.037871360778808594
[2024/09/19 22:15:13] ppocr DEBUG: rec_res num  : 19, elapsed : 0.06428980827331543
[2024/09/19 22:15:13] ppocr DEBUG: dt_boxes n

In [22]:
results = model('/content/41aZwuYCIJL.jpg')


image 1/1 /content/41aZwuYCIJL.jpg: 640x640 1 height, 1 width, 101.1ms
Speed: 3.5ms preprocess, 101.1ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)


In [38]:
results[0]

ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'depth', 1: 'height', 2: 'width'}
obb: None
orig_img: array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 